# IMPORTING LIBRARIES

In [ ]:
pip install transformers

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
pip install requests_html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [ ]:
pip install nest_asyncio

# LOADING DATASET

In [ ]:
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the English dataset from conll2012_ontonotesv5
dataset = load_dataset("conll2012_ontonotesv5", "english_v12")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10539 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1370 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [ ]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.2.1+cu121
Is CUDA enabled? True


# DATA PRE-PROCESSING

Creating a Tag Map & Tokenisation

In [ ]:
# Modify tag_map to include both integer and string representations
tag_map = {
    0: 'O', 'O': 0,
    1: 'B-PERSON', 'B-PERSON': 1, 2: 'I-PERSON', 'I-PERSON': 2,
    3: 'B-NORP', 'B-NORP': 3, 4: 'I-NORP', 'I-NORP': 4,
    5: 'B-FAC', 'B-FAC': 5, 6: 'I-FAC', 'I-FAC': 6,
    7: 'B-ORG', 'B-ORG': 7, 8: 'I-ORG', 'I-ORG': 8,
    9: 'B-GPE', 'B-GPE': 9, 10: 'I-GPE', 'I-GPE': 10,
    11: 'B-LOC', 'B-LOC': 11, 12: 'I-LOC', 'I-LOC': 12,
    13: 'B-PRODUCT', 'B-PRODUCT': 13, 14: 'I-PRODUCT', 'I-PRODUCT': 14,
    15: 'B-DATE', 'B-DATE': 15, 16: 'I-DATE', 'I-DATE': 16,
    17: 'B-TIME', 'B-TIME': 17, 18: 'I-TIME', 'I-TIME': 18,
    19: 'B-PERCENT', 'B-PERCENT': 19, 20: 'I-PERCENT', 'I-PERCENT': 20,
    21: 'B-MONEY', 'B-MONEY': 21, 22: 'I-MONEY', 'I-MONEY': 22,
    23: 'B-QUANTITY', 'B-QUANTITY': 23, 24: 'I-QUANTITY', 'I-QUANTITY': 24,
    25: 'B-ORDINAL', 'B-ORDINAL': 25, 26: 'I-ORDINAL', 'I-ORDINAL': 26,
    27: 'B-CARDINAL', 'B-CARDINAL': 27, 28: 'I-CARDINAL', 'I-CARDINAL': 28,
    29: 'B-EVENT', 'B-EVENT': 29, 30: 'I-EVENT', 'I-EVENT': 30,
    31: 'B-WORK_OF_ART', 'B-WORK_OF_ART': 31, 32: 'I-WORK_OF_ART', 'I-WORK_OF_ART': 32,
    33: 'B-LAW', 'B-LAW': 33, 34: 'I-LAW', 'I-LAW': 34,
    35: 'B-LANGUAGE', 'B-LANGUAGE': 35, 36: 'I-LANGUAGE', 'I-LANGUAGE': 36
}

def preprocess(data):
    sentences = []
    labels = []
    for example in data:
        for sentence_info in example['sentences']:
            tokens = sentence_info["words"]
            ner_tags = [tag for tag in sentence_info.get("named_entities", [])]  # Keep tags as strings
            encoded_labels = [tag_map[tag] for tag in ner_tags]  # Convert string tags to integers
            encoded_labels.extend([tag_map['O']] * (len(tokens) - len(encoded_labels)))
            sentences.append(tokens)
            labels.append(encoded_labels)
    return sentences, labels


Initialising Sentences & Labels to newly processed data

In [ ]:
train_sentences, train_labels = preprocess(dataset["train"])
valid_sentences, valid_labels = preprocess(dataset["validation"])
test_sentences, test_labels = preprocess(dataset["test"])

In [ ]:
# Assuming train_sentences is a list of sentences
vocab = {word: i+1 for i, word in enumerate(set(word for sentence in train_sentences for word in sentence))}
vocab['<pad>'] = 0

# MODEL TRAINING

Creating a Data Loader

In [ ]:
# Define PAD_TOKEN_LABEL_ID
PAD_TOKEN_LABEL_ID = tag_map['O']

class NERDataset(Dataset):
    def __init__(self, sentences, labels):
        self.sentences = sentences
        self.labels = labels

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return {'tokens': self.sentences[idx],
                'labels': self.labels[idx]}

    def collate_fn(self, batch):
        max_len = max(len(example['tokens']) for example in batch)
        padded_sentences = [torch.tensor([vocab.get(token, vocab['<pad>']) for token in example['tokens']] +
                                         [vocab['<pad>']] * (max_len - len(example['tokens'])), dtype=torch.long)
                            for example in batch]
        padded_labels = [torch.tensor([tag_map[tag] for tag in example['labels']] +
                                      [PAD_TOKEN_LABEL_ID] * (max_len - len(example['labels'])), dtype=torch.long)
                         for example in batch]

        padded_sentences = nn.utils.rnn.pad_sequence(padded_sentences, batch_first=True)
        padded_labels = nn.utils.rnn.pad_sequence(padded_labels, batch_first=True)

        return {'tokens': padded_sentences,
                'labels': padded_labels}

# Create DataLoader for training, validation, and testing data
train_dataset = NERDataset(train_sentences, train_labels)
valid_dataset = NERDataset(valid_sentences, valid_labels)
test_dataset = NERDataset(test_sentences, test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=train_dataset.collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=32, collate_fn=valid_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=test_dataset.collate_fn)


Model Training using Bi-LSTM

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_output, _ = self.lstm(embedded)
        output = self.fc(lstm_output)
        return output

# Define hyperparameters
vocab_size = len(vocab) + 1
embedding_dim = 100
hidden_dim = 128
output_dim = len(tag_map)

# Initialize the model
model = BiLSTM(vocab_size, embedding_dim, hidden_dim, output_dim)


Using ADAM optimizer & quantifying the difference between predictions & true values using CrossEntropyLoss

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop without GPU acceleration
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
criterion = criterion.to(device)

for epoch in range(10):
    model.train()
    epoch_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/10", unit="batch"):
        inputs, labels = batch['tokens'].to(device), batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        labels = labels.view(-1)
        loss = criterion(outputs.view(-1, outputs.shape[-1]), labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}/10, Loss: {epoch_loss/len(train_loader):.4f}')


Epoch 1/10: 100%|██████████| 3620/3620 [00:23<00:00, 152.12batch/s]


Epoch 1/10, Loss: 0.0635


Epoch 2/10: 100%|██████████| 3620/3620 [00:22<00:00, 160.30batch/s]


Epoch 2/10, Loss: 0.0632


Epoch 3/10: 100%|██████████| 3620/3620 [00:22<00:00, 162.53batch/s]


...

Epoch 8/10, Loss: 0.0629


Epoch 9/10: 100%|██████████| 3620/3620 [00:23<00:00, 151.20batch/s]


Epoch 9/10, Loss: 0.0627


Epoch 10/10: 100%|██████████| 3620/3620 [00:23<00:00, 151.97batch/s]

Epoch 10/10, Loss: 0.0624


Creating a function to Tag Sentences

In [ ]:
import torch
def tag_sentences(sentences, model, vocab, tag_map, device):
    model.eval()
    tagged_sentences = []
    with torch.no_grad():
        if isinstance(sentences[0], str):  # Check if it's a single sentence
            sentences = [sentences]  # Convert single sentence to a list

        for sentence in sentences:
            tokenized_sentence = [vocab.get(token, vocab['<pad>']) for token in sentence]
            tokenized_sentence = torch.tensor(tokenized_sentence, dtype=torch.long).unsqueeze(0).to(device)
            outputs = model(tokenized_sentence)
            predicted_tags = torch.argmax(outputs, dim=-1).squeeze().tolist()
            if isinstance(predicted_tags, int):
              predicted_tags = [predicted_tags]
            tagged_sentence = [tag_map[tag] for tag in predicted_tags]
            tagged_sentences.append(tagged_sentence)
    return tagged_sentences

In [ ]:
# Test the model against some sentences
text='''New York City, often simply referred to as NYC, is a bustling metropolis.
Its iconic landmarks include the Statue of Liberty, Central Park, and the Empire State Building.
Tourists flock to Times Square to experience its vibrant energy and dazzling lights.
'''
example_sentences = [text.split()]
tagged_example_sentences = tag_sentences(example_sentences, model, vocab, tag_map, device)
for sentence, tags in zip(example_sentences, tagged_example_sentences):
    print("Text:", " ".join(sentence))
    print("Predicted Tags:", tags)
    print()

Text: New York City, often simply referred to as NYC, is a bustling metropolis. Its iconic landmarks include the Statue of Liberty, Central Park, and the Empire State Building. Tourists flock to Times Square to experience its vibrant energy and dazzling lights.
Predicted Tags: ['B-GPE', 'I-GPE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-WORK_OF_ART', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'I-ORG', 'I-FAC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



In [ ]:
pip install pyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00


Using a Pre-Trained Model for Text Summarisation

In [ ]:
import PyPDF2
from transformers import BartTokenizer, BartForConditionalGeneration
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

# Function to perform text summarization
def summarize_text(text):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    model.eval()

    summaries = []
    while len(text) > 0:
      chunk = text[:1024]
      text = text[1024:]

      inputs = tokenizer(chunk, max_length=1024, return_tensors='pt', truncation=True)
      summary_ids = model.generate(inputs['input_ids'], num_beams=8, early_stopping=True)
      summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
      summaries.append(summary)

    full_summary = ' '.join(summaries)
    return full_summary

Defining a function to perform Named Entity Recognition (NER)

In [ ]:
def perform_ner(text, model, vocab, tag_map, device):
    # Tokenize the text into sentences
    sentences = text.split(". ")  # Split the text into sentences

    # Initialize a list to store NER results
    ner_results = []

    # Iterate through each sentence
    for sentence in sentences:
        # Tokenize the sentence into words
        words = sentence.split()

        # Perform NER on the sentence
        tagged_sentence = tag_sentences([words], model, vocab, tag_map, device)[0]

        # Store the words and their corresponding predicted tags
        ner_result = [(word, tag_map[tag]) for word, tag in zip(words, tagged_sentence) if tag != 0]
        ner_results.append(ner_result)

    return ner_results

Defining a function scrape the Web for YouTube links pertaining to the subject

In [ ]:
import asyncio
from requests_html import AsyncHTMLSession
import pandas as pd
import nest_asyncio

# Allow nested event loops in Jupyter notebooks
nest_asyncio.apply()

# Create an asynchronous HTML session
session = AsyncHTMLSession()

async def scrape_videos(word):
    # Define the URL with the search word
    url = f'https://www.youtube.com/results?search_query={word}'

    # Send the request to the server
    response = await session.get(url)

    # Render the JavaScript page
    await response.html.arender(sleep=3, timeout=100, keep_page=True, scrolldown=5)

    # Find the data elements
    containers = response.html.find('ytd-video-renderer.style-scope.ytd-item-section-renderer')

    # Create an empty list to store data
    data = []

    # Counter to keep track of URLs extracted
    count = 0

    # Iterate through each container to extract data
    for elem in containers:
        try:
            video_title = elem.find('yt-formatted-string.style-scope.ytd-video-renderer', first=True).text
        except AttributeError:
            video_title = 'N/A'

        try:
            video_url = 'https://www.youtube.com' + elem.find('a#video-title', first=True).attrs['href']
        except AttributeError:
            video_url = 'N/A'

        # Append the data to the list
        data.append({'title': video_title, 'url': video_url})

        # Increment the counter
        count += 1

        # Break the loop if one URL has been extracted
        if count == 1:
            break

    return data

async def search_most_viewed_videos(query):
    return await scrape_videos(query)

def generate_youtube_videos_for_ner(entities):
  youtube_data = []
  i = 0

  while i + 1 < len(entities):
        query = entities[i][0] + ' ' + entities[i + 1][0]  # Combine current and next tuple
        data = asyncio.run(search_most_viewed_videos(query))

    # Check if the data already exists in youtube_data to avoid redundancy
        for video in data:
            if video not in youtube_data:
                youtube_data.append(video)
        i += 1
  return youtube_data


In [ ]:
# Main function to orchestrate the process
def process_pdf(pdf_file_path):
    # Step 1: Extract text from the PDF file
    pdf_text = extract_text_from_pdf(pdf_file_path)

    # Step 2: Perform text summarization
    summarized_text = summarize_text(pdf_text)
    print(summarized_text)

    # Step 3: Perform NER on the summarized text
    ner_results = perform_ner(summarized_text,model,vocab,tag_map,device)
    print(ner_results)
    entities=[item for sublist in ner_results for item in sublist]
    final_entities = [(word, tag_map[tag]) for sublist in ner_results for word, tag in sublist if tag != 0]
    print(final_entities)
    youtube_data = generate_youtube_videos_for_ner(final_entities)
    return youtube_data

Loading the input pdf

In [ ]:
pdf_file_path ="Tourism in America.pdf"
youtube_data = process_pdf(pdf_file_path)

The United States offers a rich tapestry of experiences for tourists to explore. From the rugged beauty of the Grand Canyon to the sun-kissed beaches of California, the U.S. offers a myriad of attractions. In this exploration of tourism in America, we will delve into its diverse offerings, from natural wonders to cultural landm arks. America is home to an array of national parks, each offering its own unique charm. Yosemite National Park in California entices visitors with towering granite cliffs, cascading waterfalls, and ancient sequoia trees. Yellowstone National Park, spanning Wyoming, Montana, and Idaho, captivates with its geothermal wonders. In the bustling metropolis of New York City, the Statue of Liberty stands as a symbol of freedom and opportunity. In the nation's capital, Washington, D.C., visitors can marvel at iconic monuments such as the Lincoln Memorial. In New Orleans, visitors can immerse themselves in the soulful sounds of jazz on Frenchmen Street. For thrill -seeke

In [ ]:
print(youtube_data)

[{'title': 'US News LIVE: House Votes On Aid For Ukraine, Israel, Taiwan | Mike Johnson | Congress | IN18L', 'url': 'https://www.youtube.com/watch?v=Xn21MthEU4Q&pp=ygUNVW5pdGVkIFN0YXRlcw%3D%3D'}, {'title': 'Race Highlights | 2023 United States Grand Prix', 'url': 'https://www.youtube.com/watch?v=OTbaG_uTn-M&pp=ygUMU3RhdGVzIEdyYW5k'}, {'title': 'The Grand Canyon Explained | How the Earth Was Made (S2, E1) | Full Documentary | History', 'url': 'https://www.youtube.com/watch?v=LaxaE7tcwu4&pp=ygUMR3JhbmQgQ2FueW9u'}, {'title': "Grand Canyon - The Jaw-Dropping Beauty of America's National Park | Free Documentary Nature", 'url': 'https://www.youtube.com/watch?v=hNcG76DqOiI&pp=ygUOQ2FueW9uIEFtZXJpY2E%3D'}, {'title': "Wild Yosemite (Full Episode) | America's National Parks", 'url': 'https://www.youtube.com/watch?v=-Hw4iTB_Zs4&pp=ygUQQW1lcmljYSBOYXRpb25hbA%3D%3D'}, {'title': "Wild Yosemite (Full Episode) | America's National Parks", 'url': 'https://www.youtube.com/watch?v=-Hw4iTB_Zs4&pp=ygUNTmF0

In [ ]:
encountered_titles = set()
unique_youtube_data = []
for d in youtube_data:
    title = d['title']
    if title not in encountered_titles:
        encountered_titles.add(title)
        unique_youtube_data.append(d)

print(unique_youtube_data)

[{'title': 'US News LIVE: House Votes On Aid For Ukraine, Israel, Taiwan | Mike Johnson | Congress | IN18L', 'url': 'https://www.youtube.com/watch?v=Xn21MthEU4Q&pp=ygUNVW5pdGVkIFN0YXRlcw%3D%3D'}, {'title': 'Race Highlights | 2023 United States Grand Prix', 'url': 'https://www.youtube.com/watch?v=OTbaG_uTn-M&pp=ygUMU3RhdGVzIEdyYW5k'}, {'title': 'The Grand Canyon Explained | How the Earth Was Made (S2, E1) | Full Documentary | History', 'url': 'https://www.youtube.com/watch?v=LaxaE7tcwu4&pp=ygUMR3JhbmQgQ2FueW9u'}, {'title': "Grand Canyon - The Jaw-Dropping Beauty of America's National Park | Free Documentary Nature", 'url': 'https://www.youtube.com/watch?v=hNcG76DqOiI&pp=ygUOQ2FueW9uIEFtZXJpY2E%3D'}, {'title': "Wild Yosemite (Full Episode) | America's National Parks", 'url': 'https://www.youtube.com/watch?v=-Hw4iTB_Zs4&pp=ygUQQW1lcmljYSBOYXRpb25hbA%3D%3D'}, {'title': '13 California State Parks You Must Visit', 'url': 'https://www.youtube.com/watch?v=qrGshkW8cZI&pp=ygUPUGFyayBDYWxpZm9ybml

Outputting our final results

In [ ]:
print("Youtube videos that can be used as a reference for the above pdf-")
for ele in unique_youtube_data:
  print("Video Title:"+ele['title'])
  print("Video Link:"+ele['url'])



Youtube videos that can be used as a reference for the above pdf-
Video Title:US News LIVE: House Votes On Aid For Ukraine, Israel, Taiwan | Mike Johnson | Congress | IN18L
Video Link:https://www.youtube.com/watch?v=Xn21MthEU4Q&pp=ygUNVW5pdGVkIFN0YXRlcw%3D%3D
Video Title:Race Highlights | 2023 United States Grand Prix
Video Link:https://www.youtube.com/watch?v=OTbaG_uTn-M&pp=ygUMU3RhdGVzIEdyYW5k
Video Title:The Grand Canyon Explained | How the Earth Was Made (S2, E1) | Full Documentary | History
Video Link:https://www.youtube.com/watch?v=LaxaE7tcwu4&pp=ygUMR3JhbmQgQ2FueW9u
Video Title:Grand Canyon - The Jaw-Dropping Beauty of America's National Park | Free Documentary Nature
Video Link:https://www.youtube.com/watch?v=hNcG76DqOiI&pp=ygUOQ2FueW9uIEFtZXJpY2E%3D
Video Title:Wild Yosemite (Full Episode) | America's National Parks
Video Link:https://www.youtube.com/watch?v=-Hw4iTB_Zs4&pp=ygUQQW1lcmljYSBOYXRpb25hbA%3D%3D
Video Title:13 California State Parks You Must Visit
Video Link:https://